In [ ]:
# | default_exp _components.aiokafka_loop

In [ ]:
# | export
from typing import *

from os import environ
import asyncio

from aiokafka import AIOKafkaConsumer

from fast_kafka_api.logger import get_logger
from fast_kafka_api.testing import true_after, create_and_fill_testing_topic

In [ ]:
# | eval: false
# allows async calls in notebooks

import nest_asyncio
nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
logger = get_logger(__name__, level=20)
logger.debug("ok")

In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    "group.id": f"{kafka_server_url}:{kafka_server_port}_group"
}

In [ ]:
async def test_create_and_fill_testing_topic(
    *,
    kafka_config: Dict[str, str] = kafka_config,
    is_shutting_down_f: Optional[Callable[[], bool]] = None,
    seed: int = 42,
):
    msgs_sent = 317
    msgs = [f"Hello world {i:05d}".encode("utf-8") for i in range(msgs_sent)]
    
    async with create_and_fill_testing_topic(msgs, seed=seed) as topic:
        consumer = AIOKafkaConsumer(
            topic,
            bootstrap_servers=kafka_config["bootstrap.servers"],
            auto_offset_reset="earliest",
            max_poll_records=100,
        )
        logger.info(f"Consumer {consumer} created.")
        await consumer.start()
        logger.info(f"Consumer {consumer} started.")
        msgs_received = 0
        try:
            while True:
                msgs = await consumer.getmany(timeout_ms=100)
                for k, v in msgs.items():
                    msgs_received = msgs_received + len(v)
                    if len(v) > 0:
                        print(f"total received: {msgs_received}")
                if is_shutting_down_f and is_shutting_down_f():
                    break
                    
        finally:
            assert msgs_received == msgs_sent
            print(f"Total messages received: {msgs_received}")
            await consumer.stop()
            logger.info(f"Consumer {consumer} stopped.")


# asyncio.run(test_create_and_produce_testing_topic())
await test_create_and_fill_testing_topic(is_shutting_down_f=true_after(5), seed=3)

22-12-14 09:56:38.375 [INFO] fast_kafka_api.testing: create_missing_topics(['my_topic_9611984893']): new_topics = [NewTopic(topic=my_topic_9611984893,num_partitions=3)]


%4|1671011798.369|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance


22-12-14 09:56:39.382 [INFO] fast_kafka_api.testing: Producer <aiokafka.producer.producer.AIOKafkaProducer object> created.
22-12-14 09:56:39.405 [INFO] fast_kafka_api.testing: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stared.
22-12-14 09:56:39.925 [INFO] fast_kafka_api.testing: Sent messages: len(sent_msgs)=317
22-12-14 09:56:39.926 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_9611984893'})
22-12-14 09:56:39.926 [INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> created.
22-12-14 09:56:39.938 [INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic_9611984893': 3}. 
22-12-14 09:56:39.946 [INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> started.
total received: 100
total received: 112
total received: 212
total received: 223
total received: 317
Total messages received: 317
22-12-14 09:56:43.389 [INFO] __main__: Consumer